# Poking the DataCite API with Python

This notebook can be used for minting and deleting DataCite DOIs with the DataCite Member API. To execute a code cell, select it and click the play (black triangle) button in the toolbar. You can also use `Ctrl + Enter`.

Depending on the platform that you're running this notebook on, execution of code cells - especially those that make a request of the DataCite API - may take some time. Be patient, and in bulk operations you might like to get up and do some stretches or get a coffee.

## Initial setup

Create a copy of `datacite-api-config.example.json` named `datacite-api-config.json` and modify it to contain your DataCite Member API username and password. For example:
```lang=json
{
    "username" : "myusername",
    "password" : "mypassword"
}
```

By default this notebook runs on the test API.

Then execute the following code cell to perform initial setup. You swill need to execute this code cell once at the beginning of every session.

In [232]:
import csv
import datetime
import json
import os
import requests
import urllib

if not os.path.isfile('doilog.csv'):
    with open('doilog.csv', 'w') as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow([ 'timestamp', 'username', 'filename', 'doi', 'action' ])

with open('datacite-api-config.json') as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]

use_test_api = True

if use_test_api:
    api_endpoint = 'https://api.test.datacite.org/dois'
else:
    api_endpoint = 'https://api.test.datacite.org/dois'

# Singletons

## Mint a single Draft DOI

The following code cell will let you mint a single DOI in a Draft state. Draft DOIs are not findable, and can be safely deleted.

Create a `metadata.json` file that contains the metadata for your dataset following the [DataCite v4.4 metadata schema](https://schema.datacite.org/meta/kernel-4.4/). You can use [`metadata.example.json`](metadata.example.json) as a guide.

Then, execute the following code cell. If a DOI was successfully minted, it will let you know and log the creation to [`doilog.csv`](doilog.csv).

In [233]:
filename = 'metadata.json'
url = api_endpoint
headers = {
    'Content-Type': 'application/vnd.api+json',
}
data = open(filename)
response = requests.request("POST", url, auth=(username, password), data = data, headers = headers)
if (response.headers['Status'] == '201 Created'):
    doi = json.loads(response.text)['data']['id']
    timestamp = datetime.datetime.now().replace(microsecond=0).astimezone().isoformat()
    # timestamp = response.headers['Date']
    print(doi + ' minted')
    with open('doilog.csv', 'a') as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow([ timestamp, username, filename, doi , 'created' ])
else:
    print('DOI not minted')

10.80335/3w7d-6w37 minted


## Delete a single Draft DOI

Use the following code cells to delete a DOI that is in Draft state. By default the first cell will delete the last DOI you minted in this session.

If you want to delete a specific DOI, put it between the quotes in the second code cell *without* the https://doi.org/. For example, `doi = '10.80335/3vrc-qy79'`. Then, execute the second cell before executing the first cell.

In [234]:
url = api_endpoint + '/' + urllib.parse.quote_plus(doi)
response = requests.request('DELETE', url, auth=(username, password), headers=headers)
if (response.headers['Status'] == '204 No Content'):
    timestamp = datetime.datetime.now().replace(microsecond=0).astimezone().isoformat()
    # timestamp = response.headers['Date']
    print(doi + ' deleted')
    with open('doilog.csv', 'a') as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow([ timestamp, username, '', doi , 'deleted' ])
else:
    print(doi + ' not deleted')
    print(response.headers)

10.80335/3w7d-6w37 deleted


In [178]:
doi = ''

## Register, publish, or hide a single Draft DOI

The following code cell will attempt to register the DOI in the doi variable, which is automatically filled by the single minting process.

Once in Registered or Findable state, a DOI can't be set back to Draft state. This also means that once in Registered or Findable state, a DOI *cannot be delete*. This is serious, mum.

In [225]:
action = 'hide' # Set this to register, publish, or hide

payload = '{\"data\":{\"attributes\":{\"event\":\"' + action + '\"}}}'
headers = {
    'Content-Type': 'application/vnd.api+json',
}
url = api_endpoint + '/' + urllib.parse.quote_plus(doi)
response = requests.request('PUT', url, auth=(username, password), data = payload, headers=headers)
if (response.headers['Status'] == '200 OK'):
    timestamp = datetime.datetime.now().replace(microsecond=0).astimezone().isoformat()
    # timestamp = response.headers['Date']
    print(doi + ' ' + action + ' successful')
    with open('doilog.csv', 'a') as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow([ timestamp, username, '', doi , action ])
else:
    print(doi + ' not registered')
    print(response.headers)

10.80335/1337 hide successful


In [206]:
doi = '10.80335/1337'

# Bulk

## Mint bulk Draft DOIs

The following code cell will go through all files in the `bulk-mint` directory and attempt to mint a DOI for each one. The DataCite API will reject anything that is not a valid JSON file containing DataCite metadata.

Put multiple metadata.json files in the `bulk-mint` directory, and then execute the following code cell.

In [200]:
directory = 'bulk-mint'
url = api_endpoint
headers = {
    'Content-Type': 'application/vnd.api+json',
}
print('Bulk minting DOIs for files in ' + directory)
dois = []
for filename in os.listdir('bulk-mint'):
    data = open(directory + '/' + filename)
    print('Attempting to mint DOI for ' + directory + '/' + filename)
    response = requests.request("POST", url, auth=(username, password), data = data, headers = headers)
    if (response.headers['Status'] == '201 Created'):
        doi = json.loads(response.text)['data']['id']
        timestamp = datetime.datetime.now().replace(microsecond=0).astimezone().isoformat()
        # timestamp = response.headers['Date']
        print(doi + ' minted')
        dois.append(doi)
        with open('doilog.csv', 'a') as csvfile:
            logwriter = csv.writer(csvfile)
            logwriter.writerow([ timestamp, username, directory + '/' + filename, doi , 'created' ])
    else:
        print('DOI not minted')
print('Done')

Bulk minting DOIs for files in bulk-mint
Attempting to mint DOI for bulk-mint/metadata-Copy4.json
10.80335/m0m2-tf82 minted
Attempting to mint DOI for bulk-mint/metadata-Copy3.json
10.80335/eg2h-4f20 minted
Attempting to mint DOI for bulk-mint/metadata-Copy2.json
10.80335/8s05-kx79 minted
Attempting to mint DOI for bulk-mint/metadata-Copy1.json
10.80335/8372-yv95 minted
Attempting to mint DOI for bulk-mint/metadata.json
10.80335/s5t7-qy70 minted
Done


## Delete bulk Draft DOIs

The following code cell will attempt to delete all of the DOIs specified in the dois array, which is automatically filled by the bulk minting process.

If you want to specify a list of DOIs to delete, put them in the list in the second cell e.g. `dois = ['10.80335/sr34-9h64', '10.80335/5375-5t54', '10.80335/2r04-6k46', '10.80335/drgg-dp97', '10.80335/7yky-cd07']`. Then, execute the second code cell before executing the first code cell.

In [186]:
print('Bulk deleting DOIs in list')
for doi in dois:
    url = api_endpoint + '/' + urllib.parse.quote_plus(doi)
    print('Attempting to delete ' + doi)
    response = requests.request('DELETE', url, auth=(username, password), headers=headers)
    if (response.headers['Status'] == '204 No Content'):
        timestamp = datetime.datetime.now().replace(microsecond=0).astimezone().isoformat()
        # timestamp = response.headers['Date']
        print(doi + ' deleted')
        with open('doilog.csv', 'a') as csvfile:
            logwriter = csv.writer(csvfile)
            logwriter.writerow([ timestamp, username, '', doi , 'deleted' ])
    else:
        print(doi + ' not deleted')
        print(response.headers)
print('Done')

Bulk deleting DOIs in list
Attempting to delete 10.80335/rnph-3047
10.80335/rnph-3047 deleted
Attempting to delete 10.80335/k7gp-8f65
10.80335/k7gp-8f65 deleted
Attempting to delete 10.80335/qkxv-k209
10.80335/qkxv-k209 deleted
Attempting to delete 10.80335/4nzp-6a61
10.80335/4nzp-6a61 deleted
Attempting to delete 10.80335/60j6-4332
10.80335/60j6-4332 deleted
Done


In [ ]:
dois = []

In [227]:
dois

['10.80335/m0m2-tf82',
 '10.80335/eg2h-4f20',
 '10.80335/8s05-kx79',
 '10.80335/8372-yv95',
 '10.80335/s5t7-qy70']

## Register, publish, or hide bulk Draft DOIs

The following code cell will attempt to register, publish, or hide all of the DOIs specified in the dois array, which is automatically filled by the bulk minting process.

Once in Registered or Findable state, a DOI can't be set back to Draft state. This also means that once in Registered or Findable state, a DOI *cannot be deleted*. This is serious, mum.

The only option for removing a Findable DOI from the public record is to hide it.

In [229]:
action = 'hide' # Set this to register, publish, or hide

payload = '{\"data\":{\"attributes\":{\"event\":\"' + action + '\"}}}'
headers = {
    'Content-Type': 'application/vnd.api+json',
}
print('Bulk ' + action + ' DOIs in list')
for doi in dois:
    url = api_endpoint + '/' + urllib.parse.quote_plus(doi)
    print('Attempting to ' + action + ' ' + doi)
    response = requests.request('PUT', url, auth=(username, password), data = payload, headers=headers)
    if (response.headers['Status'] == '200 OK'):
        timestamp = datetime.datetime.now().replace(microsecond=0).astimezone().isoformat()
        # timestamp = response.headers['Date']
        print(doi + ' ' + action + ' successful')
        with open('doilog.csv', 'a') as csvfile:
            logwriter = csv.writer(csvfile)
            logwriter.writerow([ timestamp, username, '', doi , action ])
    else:
        print(doi + ' not deleted')
        print(response.headers)
print('Done')

Bulk hide DOIs in list
Attempting to hide 10.80335/m0m2-tf82
10.80335/m0m2-tf82 hide successful
Attempting to hide 10.80335/eg2h-4f20
10.80335/eg2h-4f20 hide successful
Attempting to hide 10.80335/8s05-kx79
10.80335/8s05-kx79 hide successful
Attempting to hide 10.80335/8372-yv95
10.80335/8372-yv95 hide successful
Attempting to hide 10.80335/s5t7-qy70
10.80335/s5t7-qy70 hide successful
Done


## Troubleshooting

If you are trying to work out why your DOI was not minted or deleted, execute the following code cell.

In [226]:
print(response.headers)
print('\n')
print(response.text)
json.loads(response.text)

{'Date': 'Fri, 20 Aug 2021 07:22:40 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Status': '200 OK', 'Cache-Control': 'max-age=0, private, must-revalidate', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'ETag': 'W/"20204c87c3e737b1467fd6b3ea6e679c"', 'X-Runtime': '0.335982', 'X-Credential-Username': 'ardcx.ardc', 'X-Request-Id': 'a35e3734-afbe-4670-956e-df414ccc6bb1', 'X-Powered-By': 'Phusion Passenger(R) 6.0.10', 'Server': 'nginx/1.14.0 + Phusion Passenger(R) 6.0.10', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, POST, PUT, PATCH, DELETE, OPTIONS', 'Access-Control-Allow-Headers': 'Accept,Access-Control-Allow-Origin,Access-Control-Expose-Headers,Access-Control-Allow-Methods,Access-Control-Allow-Headers,Authorization,Cache-Control,Content-Type,DNT,If-Modified-Since,Keep-Alive,Origin,User-Agent,X-Mx-ReqToken,X-Requested-With', 'Access-Control-Expose-Headers': 'Authorizatio

{'data': {'id': '10.80335/1337',
  'type': 'dois',
  'attributes': {'doi': '10.80335/1337',
   'prefix': '10.80335',
   'suffix': '1337',
   'identifiers': [],
   'alternateIdentifiers': [],
   'creators': [{'name': 'Matthias Liffers',
     'affiliation': [],
     'nameIdentifiers': []}],
   'titles': [{'title': 'Flooble'}],
   'publisher': 'ARDC',
   'container': {},
   'publicationYear': 2021,
   'subjects': [],
   'contributors': [],
   'dates': [],
   'language': None,
   'types': {'schemaOrg': 'ScholarlyArticle',
    'citeproc': 'article-journal',
    'bibtex': 'article',
    'ris': 'RPRT',
    'resourceTypeGeneral': 'Text'},
   'relatedIdentifiers': [],
   'sizes': [],
   'formats': [],
   'version': None,
   'rightsList': [],
   'descriptions': [],
   'geoLocations': [],
   'fundingReferences': [],
   'xml': 'PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiPz4KPHJlc291cmNlIHhtbG5zOnhzaT0iaHR0cDovL3d3dy53My5vcmcvMjAwMS9YTUxTY2hlbWEtaW5zdGFuY2UiIHhtbG5zPSJodHRwOi8vZGF0YWNpdGUub3Jn